<a href="https://colab.research.google.com/github/kaholict/Karpov-Course/blob/main/sin_analys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_excel('/content/sin_data.xlsx', sheet_name='Лист1', names=['time', 'value'], header=None)
df.head()

,time,value
0,0.00,1.708883
1,0.01,1.803424
2,1.00,2.333126
3,1.01,2.398613
4,2.00,2.681550


In [ ]:
df.value.describe()

count    500.000000
mean       1.240555
std        1.051293
min       -0.252689
25%        0.189015
50%        1.251143
75%        2.287950
max        2.721817
Name: value, dtype: float64

In [ ]:
q01 = df.value.quantile([.01])[0.01]
q25 = df.value.quantile([.25])[0.25]
q75 = df.value.quantile([.75])[0.75]
q99 = df.value.quantile([.99])[0.99]

In [ ]:
max_value = df.query('value >= @q75 and value <= @q99')
qmax75 = max_value.value.quantile([.75])[0.75]
qmax99 = max_value.value.quantile([.99])[0.99]
max_value = max_value.query('value >= @qmax75 and value <= @qmax99')
max_value.head()

,time,value
5,2.01,2.701540
30,15.00,2.718431
31,15.01,2.708192
54,27.00,2.707708
55,27.01,2.718187


In [ ]:
min_value = df.query('value >= @q01 and value <= @q25')
qmin01 = min_value.value.quantile([.01])[0.01]
qmin25 = min_value.value.quantile([.25])[0.25]
min_value = df.query('value >= @qmin01 and value <= @qmin25')
min_value.head()

,time,value
18,9.00,-0.231151
67,33.01,-0.230545
92,46.00,-0.250001
93,46.01,-0.240509
116,58.00,-0.237013


In [ ]:
min_value.value.describe()

count    28.000000
mean     -0.237034
std       0.010852
min      -0.251167
25%      -0.246650
50%      -0.239254
75%      -0.229117
max      -0.216462
Name: value, dtype: float64

In [ ]:
from scipy.misc import derivative
import math


class TaylorSeries():
    def __init__(self, function, order, center=0):
        self.center = center
        self.f = function 
        self.order = order
        self.d_pts = order*2
        self.coefficients = []

       # количество точек (order) для scipy.misc.derivative
        if self.d_pts % 2 == 0: # должно быть больше, чем порядок производной, и нечетным
            self.d_pts += 1

        self.__find_coefficients()

    def __find_coefficients(self):
        for i in range(0, self.order+1):
            self.coefficients.append(round(derivative(self.f, self.center, n=i, order=self.d_pts)/math.factorial(i), 5))

    def print_equation(self):
        eqn_string = ""
        for i in range(self.order + 1):
            if self.coefficients[i] != 0:
                eqn_string += str(self.coefficients[i]) + ("(x-{})^{}".format(self.center, i) if i > 0 else "") + " + "
        eqn_string = eqn_string[:-3] if eqn_string.endswith(" + ") else eqn_string
        print(eqn_string)

    def print_coefficients(self):
        print(self.coefficients)

    def approximate_value(self, x):
        """
           Аппроксимирует значение f(x) с помощью полинома Тейлора.
       		 x = точка аппроксимации f(x)

        """
        fx = 0
        for i in range(len(self.coefficients)):
            fx += self.coefficients[i] * ((x - self.center)**i)  # coefficient * nth term 
        return fx

    def approximate_derivative(self, x):
        """
           Приблизительно вычисляет производную функции f(x) по ее ряду Тейлора.
        	 Бесполезно, так как нам нужна производная самой функции, чтобы построить ряд Тейлора.

        """
        value = 0
        for i in range(1, len(self.coefficients)): # skip the first value (constant) as the derivative is 0
            value += self.coefficients[i] * i * ((x - self.center)**(i-1)) # differentiate each term: x^n => n*x^(n-1)
        return value

    def approximate_integral(self, x0, x1):
        """
           Вычисляет определенный интеграл функции, используя разложение в ряд Тейлора.
       		 x0 - нижний предел интегрирования
   		     x1 - верхний предел интегрирования 

        """
        
        # интегралы могут отличаться на константу, поскольку int(f(x)) = F(x) + C
        value = 0
        for i in range(len(self.coefficients)):
            value += ((self.coefficients[i] * (1/(i+1)) * ((x1 - self.center)**(i+1))) - 
                      (self.coefficients[i] * (1/(i+1)) * ((x0 - self.center)**(i+1)))) # integrate each term: x^n => (1/n+1)*x^(n+1)
        return value

    def get_coefficients(self):
        """
             Возвращает коэффициенты ряда Тейлора 
        """
        return self.coefficients

def f(x):
    return 1.23456789+1.48726359*math.sin(6.789654123*x+0.324587632) #(math.e**x)*math.sin(x)*math.cos(x)

if __name__ == '__main__':
    pts = [i/100 for i in range(0,629,1)]
    # pts = [-5, -4, -3, -2, -1, -0.1, 0, 0.1, 1, 2, 3, 4, 5]
    terms = 15
    center = 0
    precision = 10

    ts = TaylorSeries(f, terms, center)
    ts.print_coefficients()
    ts.print_equation()

    print("x\tf(x)\tApprox. f(x)\tIntegral f(x)\tDerivative f(x)")
    for x in pts:
        print("{:.10f}\t{:.10f}\t{:.10f}\t{:.10f}\t{:.10f}".format(x, f(x), ts.approximate_value(x), ts.approximate_integral(0, x), ts.approximate_derivative(x)))


[1.70888, 0.71392, -0.06083, -0.03052, 0.0013, 0.00039, -1e-05, -0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 0.0, -0.0, -0.0]
1.70888 + 0.71392(x-0)^1 + -0.06083(x-0)^2 + -0.03052(x-0)^3 + 0.0013(x-0)^4 + 0.00039(x-0)^5 + -1e-05(x-0)^6
x	f(x)	Approx. f(x)	Integral f(x)	Derivative f(x)
0.0000000000	1.7088829867	1.7088800000	0.0000000000	0.7139200000
0.0100000000	1.8034237168	1.7160130865	0.0171244756	0.7126942492
0.0200000000	1.8953430628	1.7231338240	0.0343202206	0.7114502179
0.0300000000	1.9842174447	1.7302420300	0.0515871104	0.7101879380
0.0400000000	2.0696373143	1.7373375221	0.0689250188	0.7089074418
0.0500000000	2.1512090423	1.7444201182	0.0863338178	0.7076087622
0.0600000000	2.2285567324	1.7514896368	0.1038133776	0.7062919324
0.0700000000	2.3013239534	1.7585458965	0.1213635664	0.7049569863
0.0800000000	2.3691753814	1.7655887163	0.1389842507	0.7036039581
0.0900000000	2.4317983457	1.7726179155	0.1566752953	0.7022328824
0.1000000000	2.4889042690	1.7796333139	0.1744365630	0.7008437944
0.110000000